In [3]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F
import sys
sys.path.append("..")
import library.d2lzh_pytorch as d2l

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU()
    )
    return blk

# NIN 模型

In [4]:
# 定义全局平均池化层
class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层通过将池化窗口设置为输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, x):
        # kernel_size 设置为输入的高和宽
        return F.avg_pool2d(x, kernel_size=x.size()[2:])

In [5]:

# 定义 NIN 网络
net = nn.Sequential(
    # 第一个 NIN 块
    nin_block(1, 96, kernel_size=11, stride=4, padding=0),
    nn.MaxPool2d(kernel_size=3, stride=2),
    
    # 第二个 NIN 块
    nin_block(96, 256, kernel_size=5, stride=1, padding=2),
    nn.MaxPool2d(kernel_size=3, stride=2),
    
    # 第三个 NIN 块
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2),
    
    # Dropout 层用于防止过拟合
    nn.Dropout(0.5),
    
    # 最后的 NIN 块，用于分类，类别数为 10
    nin_block(384, 10, kernel_size=3, stride=1, padding=1),
    
    # 全局平均池化层
    GlobalAvgPool2d(),
    
    # 将四维输出转换为二维，形状为 (批量大小, 10)
    nn.Flatten()
)

In [6]:
X = torch.rand(1, 1, 224, 224)  
for name, blk in net.named_children():  
    X = blk(X)  
    print(name, 'output shape: ', X.shape)

0 output shape:  torch.Size([1, 96, 54, 54])
1 output shape:  torch.Size([1, 96, 26, 26])
2 output shape:  torch.Size([1, 256, 26, 26])
3 output shape:  torch.Size([1, 256, 12, 12])
4 output shape:  torch.Size([1, 384, 12, 12])
5 output shape:  torch.Size([1, 384, 5, 5])
6 output shape:  torch.Size([1, 384, 5, 5])
7 output shape:  torch.Size([1, 10, 5, 5])
8 output shape:  torch.Size([1, 10, 1, 1])
9 output shape:  torch.Size([1, 10])


In [8]:
batch_size = 128  
# 如出现“out of memory”的报错信息,可减小小batch_size或resize  
train_iter, test_iter = d2l.load_data_fashion_mnist_in_ch5(batch_size, resize=224)  
lr, num_epochs = 0.002, 5  
optimizer = torch.optim.Adam(net.parameters(), lr=lr)  
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer,  device, num_epochs)

training on  cuda
epoch 1, loss 1.4104, train acc 0.488, test acc 0.678,  time 112.6 sec


KeyboardInterrupt: 